In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os

In [ ]:
def get_csv_data(path_logs, path_csv):
    data = pd.read_csv(path_csv, sep='\t')
    dfs = []
    for dirname, _, filenames in os.walk(path_logs):
        for filename in filenames:
            if filename.endswith('.csv'):
                file_path = os.path.join(dirname, filename).replace('\\', '/')
                df = pd.read_csv(file_path,sep=' ')
                
                row = data.loc[data['log_id'] == int(filename[:-4])] # https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
                # print(row)
                arr = row.values.tolist()
                arr.append(df)
                dfs.append(arr)
    return dfs

In [ ]:
dfs = get_csv_data('../the-attentive-cursor-dataset/logs', '../the-attentive-cursor-dataset/groundtruth.tsv')

In [ ]:
cleandf = [] #note + liste des inputs
for row in range(len(dfs)):
    cleandf.append([dfs[row][0][2],dfs[row][1]['event'].value_counts()])

In [ ]:

tab = np.array(cleandf, dtype='object')
for row in tab :
    for input in ['mouseup','focus','load','beforeunload','unload','touchend'] : #liste arbitraire des actions à enlever
        if input in row[1].keys() :
            row[1].pop(input)

#liste des actions à conserver
all_inputs = []
for row in tab :
    for input in row[1].keys() :
        if input not in all_inputs :
            all_inputs.append(input)
all_inputs.sort()
print(all_inputs)

In [ ]:
#normalisation v1 (mieux que la v2)
"""
for input in all_inputs :
    tmp = []
    for row in tab :
        if input in row[1].keys() :
            tmp.append(row[1].get(input))
    mean = np.mean(tmp)
    for row in tab :
        if input in row[1].keys() :
            row[1][input] = row[1][input]/mean

print(tab)
"""
#normalisation v3
"""
for input in all_inputs :
    max = 0
    for row in tab :
        if input in row[1].keys() :
            if row[1].get(input) > max :
                max = row[1].get(input)
    for row in tab :
        if input in row[1].keys() :
            row[1][input] = row[1][input]/max
"""

In [ ]:
tmp_all_x = [] #stock temporairement les données d'entrées
tmp_all_y = [] #stock temporairement les données de sortie

for row in tab :
    tmp_all_x.append([])
    for input in all_inputs :
        if input in row[1].keys() :
            tmp_all_x[len(tmp_all_x)-1].append(row[1].get(input))
        else :
            tmp_all_x[len(tmp_all_x)-1].append(0)
    tmp_all_y.append(row[0])

all_x = np.array(tmp_all_x) #données en entrée du modèle
all_y = np.array(tmp_all_y) #données en sortie du modèle
print(all_x)

[[0.44852941 0.0990991  0.17647059 ... 0.         0.         0.        ]
 [0.05147059 0.04054054 0.05882353 ... 0.         0.         0.        ]
 [0.08088235 0.17567568 0.05882353 ... 0.         0.         0.        ]
 ...
 [0.13970588 0.04954955 0.17647059 ... 0.         0.         0.        ]
 [0.14705882 0.09459459 0.15686275 ... 0.         0.         0.        ]
 [0.03676471 0.03603604 0.07843137 ... 0.         0.         0.        ]]


In [ ]:
#normalisation v2 (la v1 est un peu mieux)
"""
for i in range(len(all_inputs)) :
    print(i)
    tmp = []
    for row in all_x :
        tmp.append(row[i])
    mean = np.mean(tmp)
    for row in all_x :
        row[i] = row[i]/mean
    
print(all_x)
"""

#enlevage des données extrèmes (faire une normalisation avant)
"""
i = 0
bad_rows = []
for row in range(len(all_x)) :
    for element in all_x[row] :
        if element > 5 :
            i +=1
            bad_rows.append(row)
            continue

all_x = np.delete(all_x,bad_rows, axis=0)
print(all_x)
"""

'\ni = 0\nbad_rows = []\nfor row in range(len(all_x)) :\n    for element in all_x[row] :\n        if element > 5 :\n            i +=1\n            bad_rows.append(row)\n            continue\n\nall_x = np.delete(all_x,bad_rows, axis=0)\nprint(all_x)\n'

In [ ]:

inputs = keras.Input(shape=(len(all_inputs),))
x = tf.keras.layers.Dense(1, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(6, activation=tf.nn.softmax)(x)

model = keras.Model(inputs=inputs, outputs = outputs)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)



In [ ]:
history = model.fit(all_x, all_y, epochs=20, validation_split=0.1)

Epoch 1/20
82/82 [==============================] - 1s 2ms/step - loss: 1.7859 - accuracy: 0.2983 - val_loss: 1.7800 - val_accuracy: 0.2612
Epoch 2/20
82/82 [==============================] - 0s 1ms/step - loss: 1.7754 - accuracy: 0.2926 - val_loss: 1.7685 - val_accuracy: 0.2680
Epoch 3/20
82/82 [==============================] - 0s 1ms/step - loss: 1.7638 - accuracy: 0.2953 - val_loss: 1.7562 - val_accuracy: 0.2680
Epoch 4/20
82/82 [==============================] - 0s 1ms/step - loss: 1.7517 - accuracy: 0.2953 - val_loss: 1.7440 - val_accuracy: 0.2612
Epoch 5/20
82/82 [==============================] - 0s 1ms/step - loss: 1.7399 - accuracy: 0.2956 - val_loss: 1.7326 - val_accuracy: 0.2612
Epoch 6/20
82/82 [==============================] - 0s 1ms/step - loss: 1.7285 - accuracy: 0.2960 - val_loss: 1.7216 - val_accuracy: 0.2612
Epoch 7/20
82/82 [==============================] - 0s 1ms/step - loss: 1.7181 - accuracy: 0.2960 - val_loss: 1.7116 - val_accuracy: 0.2577
Epoch 8/20
82/82 [==